In [ ]:
from parse_deck import parse_event_to_deck

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

import time
import re
from tqdm import tqdm

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--start-maximized")

In [ ]:
def wait_loading_circle(timeout: int = 20):
    WebDriverWait(driver, 20).until(EC.invisibility_of_element_located((By.XPATH, "//div[@class='sk-circle-container']")))
    

In [ ]:
# parse CL event links from official website
url = "https://players.pokemon-card.com/event/result/list"
driver = webdriver.Chrome(options=chrome_options)  # options=chrome_options
driver.implicitly_wait(10) # seconds
driver.get(url)

no_page_crawl = 2
cl_events = []

# decks = {
#     deck category 1: [
#         {
#             deck_link: "",
#             deck_code: "",
#             pokemons: {},
#             tools: {},
#             supporters: {},
#             stages: {},
#             energies: {},
#             rank: 1,
#             num_people: 32,
#             date: datetime
#         }, ...
#     ],
#     deck category 2:...
# }
all_categories = ["ルギアVSTAR", "ミュウVMAX", "ジュラルドンVMAX", "ゾロア", "others"]
decks = {}

for _ in range(no_page_crawl):
    events = driver.find_elements(By.CLASS_NAME, "eventListItem")
    for event in tqdm(events):
        title = event.find_element(By.CLASS_NAME, "title")
        if "シティリーグ" in title.text:
            num_people_str = event.find_element(By.CLASS_NAME, "capacity").text
            num_people = re.findall(r'\d+', num_people_str)
            num_people = int(num_people[0]) if len(num_people) == 1 else None
            event_link = event.get_attribute("href")
            parse_event_to_deck(event_link, num_people, decks, all_categories)

    # nevigate to the next page
    driver.find_element(By.CLASS_NAME, "btn.next").click()
    wait_loading_circle()

driver.close()

In [ ]:
# decks["ルギアVSTAR"] 
len(decks)

In [ ]:
deck_name = []
for d in decks["others"]:
    lead = list(d["pokemons"].keys())[0]
    if lead not in deck_name:
        deck_name.append(lead)
        
print(deck_name)


In [ ]:
for d in decks["others"]:
    lead = list(d["pokemons"].keys())[0]
    if lead == "ゾロア":
        print(d)
        print(d["deck_link"])
        print("\n---\n")

In [ ]:
import pandas as pd
from collections import OrderedDict

target_category = "ルギアVSTAR"
df = pd.DataFrame()
# df_tool = pd.DataFrame()
# df_supporter = pd.DataFrame()
# df_stage = pd.DataFrame()
# df_energy = pd.DataFrame()

for _, deck in enumerate(decks[target_category]):
    deck_code = deck["deck_code"]  # row id
    pokemons = OrderedDict()
    pokemons["date"] = deck["date"]
    pokemons["num_people"] = deck["num_people"]
    pokemons["rank"] = deck["rank"]
    pokemons.update(deck["pokemons"])
    
    
    if _ == 0:
        df = pd.DataFrame(pokemons, index=[deck_code])
    else:
        df.loc[deck_code] = pokemons

df = df.fillna(0)

In [ ]:
df

In [ ]:
col_list = list(df)
col_list.remove("date")
col_list.remove("num_people")
col_list.remove("rank")

num_decks = df.shape[0]
print(num_decks)

num_used = df[col_list].sum(axis='rows', numeric_only=True)
num_picked = df[col_list].astype(bool).sum(axis='rows')

In [ ]:
df.loc["avg_num_used"] = {}
df.loc["pick_rate"] = {}
avg_num_used = num_used / num_picked
pick_rate = num_picked / num_decks
for col in col_list:
    df.loc["avg_num_used", col] = avg_num_used[col]
    df.loc["pick_rate", col] = pick_rate[col]

In [ ]:
num_rows = df.shape[0]
target_rows = [num_rows-1, num_rows-2]
idx = target_rows + [i for i in range(len(df)) if i not in target_rows]
df = df.iloc[idx]

In [ ]:
df

In [ ]:
pick_rate